In [1]:
# from typing import Any

# from pydantic import BaseModel
# from unstructured.partition.pdf import partition_pdf

# # Path to save images
# path = "/home/aiml/workspace/en_bn_rag/docs/"

# # Get elements
# raw_pdf_elements = partition_pdf(
#     filename=path + "FAQ_en.pdf",
#     # Using pdf format to find embedded image blocks
#     extract_images_in_pdf=False,
#     # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
#     # Titles are any sub-section of the document
#     infer_table_structure=True,
#     # Post processing to aggregate text once we have the title
#     chunking_strategy="by_title",
#     # Chunking params to aggregate text blocks
#     # Attempt to create a new chunk 3800 chars
#     # Attempt to keep chunks > 2000 chars
#     # Hard max on chunks
#     max_characters=10000,
#     new_after_n_chars=7000,
#     combine_text_under_n_chars=5000,
#     image_output_dir_path=path,
# )

In [2]:
def generate_markdown_table(data):
    """Generate Markdown table from a nested list."""
    markdown = ""
    headers = data[0]
    markdown += "| " + " | ".join([str(h) if h else "" for h in headers]) + " |\n"
    markdown += "|" + "|".join(["---" if h else "" for h in headers]) + "|\n"

    for row in data[1:]:
        markdown += "| " + " | ".join([str(cell) if cell else "" for cell in row]) + " |\n"

    return markdown

In [3]:
import pdfplumber
table_html =[]
settings = {
   "vertical_strategy": "lines",
   "horizontal_strategy": "lines",
   "snap_y_tolerance": 10, 
}

# with pdfplumber.open("/home/aiml/workspace/en_bn_rag/docs/FAQ_en.pdf") as pdf:
#     for page in enumerate(pdf.pages):
        # im = page.to_image(resolution =150)
        # im.reset().debug_tablefinder(settings)
        
        # table = page.extract_table()
        # print(page)
        # table_html.append(convert_to_html_table(table))




In [4]:

# table = P0.extract_table()
# convert_to_html_table(table)

In [5]:

class extract_data():
    def __init__(self):
        self.table_settings = {
                            "vertical_strategy": "lines_strict",
                            "horizontal_strategy": "lines_strict",
                            "snap_y_tolerance": 5,
                            "intersection_x_tolerance": 15,
                        }
    def generate_markdown_table(self, data):
        """Generate Markdown table from a nested list."""
        markdown = ""
        headers = data[0]
        markdown += "| " + " | ".join([str(h) if h else "" for h in headers]) + " |\n"
        markdown += "|" + "|".join(["---" if h else "" for h in headers]) + "|\n"

        for row in data[1:]:
            markdown += "| " + " | ".join([str(cell) if cell else "" for cell in row]) + " |\n"

        return markdown

    def extract_table_from_page(self, page):

        html_tables = []
        tables = page.extract_tables(self.table_settings)
        if tables != None:
            for table in tables:
                print(table)
                html_tables.append(self.generate_markdown_table(table))
            return html_tables
        
    def convert_image(self, page):
        img = page.to_image()
        img.show()

    def extract_text_from_page(self, page):
        page_width, page_height = page.width, page.height
        non_table_text = []
        tables = page.find_tables(self.table_settings)

        table_bboxes = [table.bbox for table in tables]
                    # Extract non-table text
        if table_bboxes:
            # Invert table bounding boxes to define non-table regions
            non_table_regions = []
            current_top = 0  # Start from the top of the page

            for bbox in sorted(table_bboxes, key=lambda x: x[1]):  # Sort by top coordinate (y0)
                top, bottom = bbox[1], bbox[3]  # y0 (top), y1 (bottom)

                # Add region above the table
                if current_top < top:
                    non_table_regions.append((0, current_top, page_width, top))
                current_top = bottom  # Update current top to the bottom of the current table

            # Add remaining region below the last table
            if current_top < page_height:
                non_table_regions.append((0, current_top, page_width, page_height))

            # Extract text from non-table regions
            page_text = []
            for region in non_table_regions:
                cropped_page = page.within_bbox(region)
                cropped_text = cropped_page.extract_text()
                if cropped_text:
                    page_text.append(cropped_text)

            if page_text:
                non_table_text.append(page_text)
        else:
            # If no tables, extract full page text
            text = page.extract_text()
            if text:
                non_table_text.append(text)
        
        return non_table_text


In [6]:
pdf = pdfplumber.open("/home/aiml/workspace/en_bn_rag/docs/FAQ_en.pdf")
extracted_text = []
extracted_table = []
extraction_class = extract_data()
for page in pdf.pages:
    tables =  extraction_class.extract_table_from_page(page)
    text = extraction_class.extract_text_from_page(page)
    extracted_text.append(text[0])
    if len( tables) >0:
        for table in tables:
            extracted_table.append(table)
    

[['Savings\nDeposit', 'Special Notice Deposit(SND)', None, 'Fixed Deposit', None], [None, '< 1.00 crore', '1.25', '3 months but<6\nmonths', '3.50-8.00'], [None, '1.00 crore but\n<25.00 crore', '1.50', '6 months but<1\nyear', '3.75-8.25'], [None, '25.00 crore but\n<50.00 crore', '1.75', '1 year but<2\nyears', '4.00-8.25'], [None, '50.00 crore but\n<100.00 crore', '2.00', '2 years but<3\nyears', '4.00-8.50'], ['-2.50', '100.00 crore\nand above', '2.50', '3 years and\nabove', '4.00-8.75']]
[['Particulars', 'Charge/FeeAmount'], ['ClosingCharges', 'BDT200.00'], ['ExciseDuty&VAT', 'AsperGovernmentRules']]
[['AccountStatementCharge', None], ['RegularStatement', 'Free'], ['Supply of Duplicate Statement ofAccount\nonDemand', 'Withinoneyear:Tk500/-\nMorethanoneyear:Tk500/-+Tk.500foreachyear.'], ['Balance Confirmation Certificate (Yearly&\nHalfYearly)', 'Issuance of Balance Certificates more than two\ntimesTk.200/-toberealized'], ['Authentication of client statement at the\nrequestofotherbanks', 

In [7]:
len(extracted_table)

11

In [8]:
# from unstructured.partition.pdf import partition_pdf

# fname = "/home/aiml/workspace/en_bn_rag/docs/FAQ_en.pdf"

# elements = partition_pdf(filename=fname,
#                          infer_table_structure=True,
#                          strategy='hi_res',
#                          combine_text_under_n_chars=3000
#            )

# tables = [el for el in elements if el.category == "Table"]

# print(tables[0].text)
# print(tables[0].metadata.metadata.text_as_html)

In [9]:
# class Element(BaseModel):
#     type: str
#     text: Any


# # Categorize by type
# categorized_elements = []
# for element in raw_pdf_elements:
#     if "unstructured.documents.elements.Table" in str(type(element)):
#         categorized_elements.append(Element(type="table", text=str(element)))
#     elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
#         categorized_elements.append(Element(type="text", text=str(element)))

# table_elements = [el.metadata.text_as_html for el in raw_pdf_elements if el.category =="Table"]
# print(len(table_elements))

# # Text
# text_elements = [e for e in categorized_elements if e.type == "text"]
# print(len(text_elements))

In [10]:
extracted_text

["CASA Account\n1. What is a current account?\nAns. A Current Account is designed for businesses to manage daily transactions\nlike cheque issuance, deposits, fund transfers, withdrawals, etc. Every individual\nwho wants to start a business must open a Current Account with a bank.\n2. What is a savings account?\nAns. Savings Account is the most basic Bank Account type that you can open\nwith any bank or financial institution. A Savings Account lets you park your idle\nmoney, thus keeping it safe. Depositing funds is the best option rather than hiding\nthem in your home’s locker, where the chances of theft are more. Whenever you\nopen a Savings Account, the banks generally give you a Debit Card that offers\neasy access to money through ATM withdrawal, plus you can also get your hands\non offers by top brands.\n3. What is the benefit of creating a current account?\nAns. 5 benefits of a Current Account:\n1. Business Transactions: A current account is specifically designed\nfor businesses 

In [11]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [12]:
# Prompt text
prompt_text = """You are an assistant tasked with summarizing text. \
    When given a question-and-answer passage, your role is to create a concise summary that captures all relevant contextual details from the text. \
    Question-and-answer content: {element}"""
prompt_text = ChatPromptTemplate.from_template(prompt_text)

# Prompt table
prompt_table = """You are an assistant responsible for summarizing tables. \
    When provided with the MARKDOWN table, your task is to create a concise summary that captures all the information presented in the table. \
    MARKDOWN table: {element}"""
prompt_table = ChatPromptTemplate.from_template(prompt_table)


# Summary chain
model = ChatOllama(model="llama3.2:3b")
summarize_chain_text = {"element": lambda x: x} | prompt_text | model | StrOutputParser()
summarize_chain_table = {"element": lambda x: x} | prompt_table | model | StrOutputParser()

/tmp/ipykernel_1720/940513099.py:15: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama3.2:3b")


In [13]:
# Apply to text
# texts = [i.text for i in text_elements if i.text != ""]
# text_summaries = summarize_chain_text.batch(texts, {"max_concurrency": 5})

In [14]:
# Apply to tables
# tables = [i for i in table_elements]
table_summaries = summarize_chain_table.batch(extracted_table)

In [17]:
table_summaries[6]

'Here is a concise summary of the table:\n\nThe service provides three types of transactions: Send Money, Cash Out, and Cash In. \n\n- Daily transaction limits and counts are:\n    - Send Money: $25,000 with 50 transactions.\n    - Cash Out: $25,000 with 15 transactions.\n    - Cash In: $30,000 with 15 transactions.\n\n- Monthly transaction limits and counts are:\n    - Send Money: $100,000 with 100 transactions.\n    - Cash Out: $150,000 with 150 transactions.\n    - Cash In: $200,000 with 150 transactions.'

'Here is a concise summary of the table:\n\n**Interest Rate Summary**\n\n* The interest rate varies based on the amount deposited:\n + For deposits up to ₹1 crore, the rates are as follows:\n - Less than ₹25 lakh: 1.25%\n - ₹25 lakh to ₹50 crore: 1.75%\n - Above ₹50 crore: 2.00%\n* For deposits above ₹100 crore, the rates are as follows:\n + Between ₹50 crore and ₹100 crore (inclusive): 4.00-8.50%\n + ₹100 crore and above: 4.00-8.75%\n\nAdditionally, there is a special notice deposit (SND) scheme with interest rates as follows:\n\n* For deposits up to ₹1 crore, the rates are as follows:\n - Less than ₹25 lakh: \n - ₹25 lakh to ₹50 crore: 3 months but < 6 months: 3.50-8.00%\n - Above ₹50 crore but below ₹100 crore: 6 months but < 1 year: 3.75-8.25%\n* For deposits above ₹100 crore, the rates are:\n + Between ₹50 crore and ₹100 crore (inclusive): 4.00-8.50%\n + ₹100 crore and above: 4.00-8.75%'


# Add to vectorstore

In [58]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.documents import Document

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="summaries", embedding_function=GPT4AllEmbeddings()
)

# The storage layer for the parent documents
store = InMemoryStore()  # <- Can we extend this to images
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [59]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# Prompt template
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Option 1: LLM
model = ChatOllama(model="llama2:13b-chat")
# Option 2: Multi-modal LLM
# model = LLaVA

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)